# Deutsch Algorithm

This kata introduces you to Deutsch algorithm - the single-qubit variant of Deutsch–Jozsa algorithm, one of the most famous educational algorithms in quantum computing.

**This kata covers the following topics:**

- The problem solved by Deutsch algorithm and the classical solution to it
- Single-qubit phase oracles (for a more detailed introduction to phase oracles, see Oracles kata)
- Deutsch algorithm
- Implementing oracles and end-to-end Deutsch algorithm in Qiskit

**What you should know to start working on this kata:**

- Basic single-qubit gates
- Quantum measurements

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister
from test_DeutschAlgorithm import exercise

## The problem solved by Deutsch algorithm

You are given a classical function that takes one bit as an input and returns one bit: $f(x): \{0, 1\} \to \{0, 1\}$. You are guaranteed that the function $f$ is

- either *constant* (returns the same value for all inputs) 
- or *variable* (returns different values for different inputs). 

The task is to figure out whether the function is constant or variable. In other words, you need to decide whether $f(0) = f(1)$ (which is the same as the function being constant for single-bit functions).

**Examples**

- $f(x) \equiv 0$ or $f(x) \equiv 1$ are constant functions (and they are actually the only constant functions in existence).
- $f(x) = x$ and $f(x) = 1 - x$ are the only variable functions (and they are the only variable functions for single-bit functions).

If you solve this problem classically, how many calls to the given function will you need? 

The first function call will give you no information - regardless of whether it returns $0$ or $1$, the function could still be constant or variable.
You'll need to call the function a second time to evaluate its return values for both possible inputs to be able to check whether these values are equal.
This means that the classical solution requires **2** function calls.

What about the quantum scenario?

## Quantum functions: single-qubit oracles

In the quantum scenario, the classical function you're working with is implemented as a quantum oracle - a "black box" operation used as input to another algorithm. This operation is implemented in a way which allows performing calculations not only on individual inputs, but also on superpositions of inputs. 

The oracle has to act on quantum states instead of classical values. 
To enable this, integer input $x$ is represented as a single-qubit state $\ket{x}$.

The type of oracles used in this tutorial are called *phase oracles*. A phase oracle $U_f$ encodes the value of the classical function $f$ it implements in the phase of the qubit state as follows:

$$U_f \ket{x} = (-1)^{f(x)} \ket{x}$$

In this case $f$ can return only two values, 0 or 1, which result in no phase change or multiplication by a relative phase $-1$, respectively.

The effect of such an oracle on any single basis state isn't particularly interesting: it just adds a global phase which isn't something you can observe. However, if you apply this oracle to a *superposition* of basis states, its effect becomes noticeable. 
Remember that quantum operations are linear: if you define the effect of an operation on the basis states, you'll be able to deduce its effect on superposition states (which are just linear combinations of the basis states) using its linearity.

There are only four single-bit functions.

1. $f(x) \equiv 0$

This is the easiest function to implement: if $f(x) \equiv 0$, 

$$U_f \ket{x} \equiv (-1)^0 \ket{x} = \ket{x}$$

This means that $U_f$ is an identity - a transformation which does absolutely nothing! 

2. $f(x) \equiv 1$

The second constant function is slightly trickier: if $f(x) \equiv 1$,

$$U_f \ket{x} \equiv (-1)^1 \ket{x} = - \ket{x}$$

Now $U_f$ is a negative identity, that is, a transformation which applies a global phase of $-1$ to the state. 
A lot of algorithms just ignore the global phase accumulated in them, since it isn't observable. 
However, if you want to be really meticulous, you can use a sequence of several single-qubit gates that, multiplied, produce a negative identity. One way to do it is to use the sequence of gates $ZXZX$.

3. $f(x) = x$

$$U_f \ket{x} = (-1)^{f(x)} \ket{x} = (-1)^{x} \ket{x}$$

This means that you don't need to do anything if the qubit is in the $\ket{0}$ state, and apply a phase of $-1$ if it is in the $\ket{1}$ state. This is exactly the effect of the $Z$ gate!

4. $f(x) = 1 - x$

You can combine the analysis for two previous cases to figure out the implementation for this phase oracle too.

In the following demo, you'll see how to implement the first three one-bit functions as quantum oracles, and observe their effect on a qubit state.
After that, you'll try to implement the oracle for the fourth function on your own!

In [ ]:
from math import acos
from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator

def oracle_zero(circ: QuantumCircuit, q: QuantumRegister) -> None:
    # Do nothing
    ...

def oracle_one(circ: QuantumCircuit, q: QuantumRegister) -> None:
    # Apply global phase -1
    circ.x(q)
    circ.z(q)
    circ.x(q)
    circ.z(q)

def oracle_x(circ: QuantumCircuit, q: QuantumRegister) -> None:
    # Apply Z gate
    circ.z(q)

# Create the simulator instance to add save_statevector method to QuantumCircuit
simulator = AerSimulator(method='statevector')

for (oracle, name) in [
    (oracle_zero, 'f(x) = 0'),
    (oracle_one, 'f(x) = 1'),
    (oracle_x, 'f(x) = x')
]:
    # Define the quantum circuit
    q = QuantumRegister(1)
    circ = QuantumCircuit(q)
    circ.ry(2 * acos(0.6), q)

    # Apply the oracle for f(x) = x; this multiples the amplitude of the |1❭ basis state by -1
    # Experiment with using other oracles to see their behavior!
    oracle(circ, q)
    circ.save_statevector()

    # Run the simulation and get the results
    res = simulator.run(circ, shots=1).result().get_statevector()
    print(f'Applying {name} oracle to state 0.6|0❭ + 0.8|1❭: {res.data}')


## Exercise 1. Oracle for f(x) = 1 - x

**Inputs:** 

1. A quantum circuit.
2. A qubit in state $\ket{\psi} = \alpha \ket{0} + \beta \ket{1}$, represented as `QuantumRegister` of length 1.

**Goal:** 
Apply the phase oracle $U_f$ for $f(x) = 1 - x$ to the qubit.
That is, apply a relative phase $(-1)^{f(x)}$ to each basis state $\ket{x}$.

<details>
<summary><strong>Need a hint?</strong></summary>
You can represent the effect of the oracle as

$$U_f \ket{x} = (-1)^{1-x} \ket{x} = (-1) \cdot (-1)^x \ket{x}$$

Can you get this effect by combining some of the previous oracles implementations?
</details>

In [ ]:
@exercise
def oracle_one_minus_x(circ: QuantumCircuit, q: QuantumRegister) -> None:
    # Write your code here
    ...

## Solving the problem: Deutsch algorithm

Now let's return to the problem of figuring out whether the given function is constant or variable for single-bit functions.
What can we do if we are given a quantum oracle $U_f$ implementing the function $f(x)$?

There are two possible inputs to the function, $\ket{0}$ and $\ket{1}$. Let's see what happens if you apply the oracle to their superposition:

$$U_f \left( \frac{1}{\sqrt2} \big( \ket{0} + \ket{1} \big) \right) 
= \frac{1}{\sqrt2} \big( U_f \ket{0} + U_f \ket{1} \big) 
= \frac{1}{\sqrt2} \big( (-1)^{f(0)} \ket{0} + (-1)^{f(1)} \ket{1} \big)$$

- If $f(0) = f(1)$, the relative phases of the two basis states are the same, and the resulting state is $\ket{+} = \frac{1}{\sqrt2} \big( \ket{0} + \ket{1} \big)$ (up to a global phase). 
- If $f(0) \neq f(1)$, the relative phases of the two basis states differ by a factor of $-1$, and the resulting state is $\ket{-} = \frac{1}{\sqrt2} \big( \ket{0} - \ket{1} \big)$ (up to a global phase). 

Now, the states $\ket{+}$ and $\ket{-}$ can be distinguished using measurement: if you apply the H gate to each of them, you'll get $H\ket{+} = \ket{0}$ if $f(0) = f(1)$, or $H\ket{-} = \ket{1}$ if $f(0) \neq f(1)$. This means that one oracle call doesn't let you calculate both $f(0)$ and $f(1)$, but it allows you to figure out whether $f(0) = f(1)$!

Overall, the algorithm is very straightforward:

1. Start with a qubit in the $\ket{0}$ state.
2. Apply the $H$ gate to the qubit.
3. Apply the oracle.
4. Apply the $H$ gate to the qubit again.
5. Measure the qubit: if it's in the $\ket{0}$ state, the function is constant, otherwise it's variable.

Note that this algorithm requires only **1** oracle call, and always produces the correct result (the algorithm is deterministic).

## Visualizing Deutsch algorithm

You can follow the steps of the algorithm for the constant and the balanced scenarios using a neat visualization. Since Deutsch algorithm deals only with states with real amplitudes, you can map all states on the unit circle, and follow the state evolution through the steps.

1. Start with a qubit in the $\ket{0}$ state and apply the $H$ gate to the qubit.
   <center><img src="./media/Plus_state.svg"></img></center>

2. Apply the oracle.  
   Here, the difference between the two scenarios becomes noticeable. In the constant scenario, $\ket{0}$ and $\ket{1}$ states get the same phase (either $1$ or $-1$), so the state remains the same or acquires a global phase of $-1$, which is physically the same state. In the variable scenario, zero and one states get different phases, so the state changes!
   <center><img src="./media/Apply_oracle.svg"></img></center>

3. Apply the $H$ gate to the qubit again.
   Now, you get the $\ket{0}$ state for both constant scenarios and the $\ket{1}$ state for both variable scenarios!
   <center><img src="./media/Apply_hadamard.svg"></img></center>


## Exercise 2. Implement Deutsch algorithm

**Input:** 
The "black box" oracle the implements $f(x)$, defined as a function that takes two arguments, `QuantumCircuit` and `QuantumRegister` (same as the oracles we've seen earlier in this kata).

**Goal:** 
Return `True` if the function is constant ($f(0) = f(1)$), or `False` if it is variable ($f(0) \neq f(1)$).
You can use only one oracle call!

In [ ]:
@exercise
def is_function_constant(oracle: callable) -> None:
    # Write your code here
    ...

# Conclusion

Congratulations! In this kata you learned Deutsch algorithm.

- Deutsch algorithm is the smallest example of a quantum algorithm that allows to answer a question about a function in fewer queries than its classical counterpart: one query to a quantum oracle versus two queries to a classical function.
- Quantum oracles don't allow you to evaluate the function on all inputs at once! Instead, Deutsch algorithm finds a clever way to aggregate information about both function values into a single bit that indicates whether they are equal or not.

Next, you will learn about the more general case of this problem and the algorithm to solve it in the "Deutsch-Jozsa Algorithm" kata.